# Topic Modelling using LDA

> ☝️Before moving on with this notebook, ensure that you have the following packages installed into your conda environment that runs this notebook: `conda install gensim=3.8.3 spacy matplotlib numpy pandas nltk`

**Overview**: In this notebook, we will explore topic modelling using Latent Dirichlet Allocation (LDA) on a corpus of Geological Survey of Western Australia reports.

⚠️This notebook takes approximately 50 minutes to execute all cells. The visualisations of the topic models is the culprit.

## Table of Contents
1. [Loading GSWA Corpus](#load_corpus)
2. [Corpus Pre-processing](#preprocessing)
3. [Training LDA Model](#training)
4. [Interactive Visualisation](#visualisation)
5. [Additional Content: LDA Model Hyperparameter Tuning](#tuning)

### Import Dependencies
- [re](https://docs.python.org/3/library/re.html) - library that we will use for performing regular expressions
- [numpy](https://numpy.org/) - library that we will use for helping visualise results
- [pandas](https://pandas.pydata.org/) - library that we will use to help handling our corpus
- [gensim](https://radimrehurek.com/gensim/) - library that we will use to perform topic modelling
- [spacy](https://spacy.io/) - library that we will use to perform lemmatization
- [nltk](https://www.nltk.org/) - library that we will use to pre-process our corpus
- [pyLDAvis](https://pyldavis.readthedocs.io/en/latest/readme.html) - library that we will use to interactively visualise results

In [1]:
# We will install pyLDAvis to help us visualise our topic models results
!pip install pyLDAvis

In [2]:
import re
from pprint import pprint
import logging
import warnings
import zipfile
import json
from typing import List

import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy

import pyLDAvis
import matplotlib.pyplot as plt

### Set-up notebook environment and helper functions

In [3]:
# Download NLTK stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\22917746\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)

## Import the GSWA dataset <a name="load_corpus"></a>

The 20-Newsgroups dataset contains about 6k data posts from 1,135 GSWA reports. 

The dataset can be imported using `pandas.read_json` and we can see the dataset has 3 columns. The `target-name` is the category or topics that the news is manually assigned to. 

We choose to use this dataset because it has pre-assigned categories according to topics, so our clustering results can be readily compared. In real world data, this is not always easily available. 

In [5]:
corpus_path = r"../data/wamex_xml.zip"

In [6]:
corpus = list()
with zipfile.ZipFile(corpus_path, "r") as z:
    for filename in z.namelist():
        with z.open(filename) as f:
            # load the json file
            # The resulting `content` is a list
            content = json.loads(f.read()) 
            # Convert content to a string   
            content = "".join(content)
            # Add to the data list
            corpus.append(content)

In [7]:
pprint(corpus[:1])

['Combined Reporting Application Pending Status: Sheet 1:250 000: Kalgoorlie '
 '(SH 51-09) Sheet 1:100 000: Kalgoorlie (3136) Project Operator: Cazaly '
 'Resources Limited Author: M Watts Date: September 2006 Distribution: '
 '1.Department of Industry and Resources 2.Cazaly Resources Limited Cazaly '
 'Resources Limited September 2006 2 Figure 2: Castle Hill Project Tenement '
 'Location Plan, 1: 75 000 scale 6 Figure 3: Regional Geology with Exploration '
 'Index Map, 1:75 000 scale 8Cazaly Resources Limited September 2006 3 1.0 '
 'SUMMARY The Combined Mineral Exploration Report on the Castle Hill Project '
 'details exploration activities undertaken by Cazaly Resources Limited during '
 'the reporting period from 7 July 2005 to 6 July 2006.The reporting group '
 'known as Castle Hill comprises nine granted prospecting licences.The project '
 'area covers some 1250 hectares.Cazaly is the registered holder of the Castle '
 'Hill Project tenements.Cazalys exploration activities durin

## Pre-processing <a name="preprocessing"></a>

Before performing topic modelling, we need to pre-process our corpus. Here we will remove unnecessary information using regular expressions via `re`.

In [8]:
# Create set of English stopwords and include domain-specific words 
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [9]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# If an issue occurs here it's because en_core_web_sm needs to be installed
# Use: !pip install -U spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [10]:
def lemmatize_doc(tokenized_doc: List[str], allowed_postags:list=['NOUN', 'ADJ', 'VERB', 'ADV']) -> List[str]:
    '''Document lemmatization using SpaCy
        
        Ref:
        - https://spacy.io/api/annotation
    '''
    
    spacy_doc = nlp(" ".join(tokenized_doc))
    doc_out = []
    for token in spacy_doc:
        if token.pos_ in allowed_postags:
            doc_out.append(token.lemma_)
        else:
            doc_out.append(token.text)
    return doc_out

In [11]:
def preprocess_and_tokenize_corpus(corpus: List[str],
                      stopwords: List[str],
                      allowed_postags:list=['NOUN', 'ADJ', 'VERB', 'ADV']) -> List[List[str]]:
    '''
    Preprocesses a corpus of texts. 
    '''
    
    preprocessed_corpus = []
    for doc in corpus:
        if len(" ".join(doc))==0 or len(" ".join(doc))>1000000:
            pass
        else:
            # Remove Figure No.'s
            doc = re.sub('Figure\s+\d+', '', doc)
            # Remove all numbers
            doc = re.sub('\d+', '', doc)
            # Remove new line characters
            doc = re.sub('\s+', ' ', doc)
            # Remove distracting single quotes
            doc = re.sub("\'", '', doc)
            # Run Gensim's simple_preprocesser - this tokenizes the document
            doc = simple_preprocess(str(doc), deacc=True)
            # Remove stopwords
            doc = [word for word in doc if word not in stopwords]
            # Lemmatize
            doc = lemmatize_doc(doc)
            
            preprocessed_corpus.append(doc)
    return preprocessed_corpus

In [12]:
# Pre-process and tokenize corpus
tokenized_corpus = preprocess_and_tokenize_corpus(corpus=corpus, stopwords=stop_words)

In [13]:
# View part of pre-processed corpus
pprint(tokenized_corpus[:1])

[['combine',
  'reporting',
  'application',
  'pende',
  'status',
  'sheet',
  'kalgoorlie',
  'sh',
  'sheet',
  'kalgoorlie',
  'project',
  'operator',
  'cazaly',
  'resources',
  'limit',
  'author',
  'watt',
  'date',
  'september',
  'distribution',
  'department',
  'industry',
  'resources',
  'cazaly',
  'resources',
  'limited',
  'cazaly',
  'resources',
  'limit',
  'september',
  'castle',
  'hill',
  'project',
  'tenement',
  'location',
  'plan',
  'scale',
  'regional',
  'geology',
  'exploration',
  'index',
  'map',
  'scale',
  'cazaly',
  'resources',
  'limit',
  'september',
  'summary',
  'combine',
  'mineral',
  'exploration',
  'report',
  'castle',
  'hill',
  'project',
  'detail',
  'exploration',
  'activity',
  'undertake',
  'cazaly',
  'resource',
  'limit',
  'reporting',
  'period',
  'july',
  'july',
  'report',
  'group',
  'know',
  'castle',
  'hill',
  'comprise',
  'nine',
  'grant',
  'prospecting',
  'licence',
  'project',
  'area',
  

### Creating bigram and trigrams - Gensim’s Phrases model

Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.

Some examples in our example are: ‘front_bumper’, ‘oil_leak’, ‘maryland_college_park’ etc.

Gensim’s Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are min_count and threshold. The higher the values of these param, the harder it is for words to be combined to bigrams.

In this notebook, we focus on building a bigram model. However the code that is required to build a trigram model is made available (it just needs to be converted to a code cell).

In [14]:
bigram = gensim.models.Phrases(tokenized_corpus, min_count=5, threshold=100) # higher threshold fewer phrases.

Faster way to get a sentence clubbed as a trigram/bigram

In [15]:
bigram_mod = gensim.models.phrases.Phraser(bigram)

View example of bigram model

In [16]:
# See example of bigram model applied to pre-processed corpus
pprint(bigram_mod[tokenized_corpus[0]])

['combine',
 'reporting',
 'application',
 'pende',
 'status',
 'sheet',
 'kalgoorlie',
 'sh',
 'sheet',
 'kalgoorlie',
 'project',
 'operator',
 'cazaly_resources',
 'limit',
 'author',
 'watt',
 'date',
 'september',
 'distribution',
 'department_industry',
 'resources',
 'cazaly_resources',
 'limited',
 'cazaly_resources',
 'limit',
 'september',
 'castle_hill',
 'project',
 'tenement',
 'location',
 'plan',
 'scale',
 'regional',
 'geology',
 'exploration',
 'index',
 'map',
 'scale',
 'cazaly_resources',
 'limit',
 'september',
 'summary',
 'combine',
 'mineral',
 'exploration',
 'report',
 'castle_hill',
 'project',
 'detail',
 'exploration',
 'activity',
 'undertake',
 'cazaly',
 'resource',
 'limit',
 'reporting_period',
 'july',
 'july',
 'report',
 'group',
 'know',
 'castle_hill',
 'comprise',
 'nine',
 'grant',
 'prospecting_licence',
 'project',
 'area',
 'cover',
 'hectare',
 'cazaly_registered',
 'holder',
 'castle_hill',
 'project',
 'tenement',
 'cazalys',
 'exploratio

Convert tokenized corpus into bi-gram phrases - this will allow the topic model to capture domain specific phrases in addition to individual words

In [17]:
bigram_corpus = [bigram_mod[tokenized_doc] for tokenized_doc in tokenized_corpus]

To perform topic modelling with LDA we need to convert the corpus of pre-processed texts into a bag-of-words (bow) representation with count frequencies.

In [18]:
# Create a dictionary that maps each word to a unique id
id2word = corpora.Dictionary(bigram_corpus)

In [19]:
id2word[162]

'prospecting_licence'

Convert the bigram corpus into a [bag-of-words (bow) representation](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

In [20]:
bow_corpus = [id2word.doc2bow(text) for text in bigram_corpus]

In [21]:
# Review a single document in bow format; (index, count frequency) 
print(bow_corpus[:1])

[[(0, 1), (1, 3), (2, 1), (3, 8), (4, 1), (5, 2), (6, 2), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 17), (13, 3), (14, 1), (15, 1), (16, 4), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 13), (26, 5), (27, 2), (28, 7), (29, 3), (30, 2), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 5), (37, 2), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 2), (45, 1), (46, 4), (47, 2), (48, 1), (49, 1), (50, 1), (51, 3), (52, 2), (53, 4), (54, 1), (55, 1), (56, 3), (57, 1), (58, 3), (59, 1), (60, 1), (61, 4), (62, 1), (63, 1), (64, 3), (65, 1), (66, 1), (67, 13), (68, 1), (69, 1), (70, 1), (71, 3), (72, 1), (73, 2), (74, 2), (75, 2), (76, 3), (77, 1), (78, 1), (79, 1), (80, 5), (81, 4), (82, 3), (83, 3), (84, 1), (85, 4), (86, 2), (87, 1), (88, 1), (89, 2), (90, 2), (91, 1), (92, 1), (93, 2), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 4), (104, 7), (105, 1), (106, 1), (107, 1), (108, 2), (109, 1), (110,

In [22]:
# View the corresponding corpus with bigrams
print(bigram_corpus[0])

['combine', 'reporting', 'application', 'pende', 'status', 'sheet', 'kalgoorlie', 'sh', 'sheet', 'kalgoorlie', 'project', 'operator', 'cazaly_resources', 'limit', 'author', 'watt', 'date', 'september', 'distribution', 'department_industry', 'resources', 'cazaly_resources', 'limited', 'cazaly_resources', 'limit', 'september', 'castle_hill', 'project', 'tenement', 'location', 'plan', 'scale', 'regional', 'geology', 'exploration', 'index', 'map', 'scale', 'cazaly_resources', 'limit', 'september', 'summary', 'combine', 'mineral', 'exploration', 'report', 'castle_hill', 'project', 'detail', 'exploration', 'activity', 'undertake', 'cazaly', 'resource', 'limit', 'reporting_period', 'july', 'july', 'report', 'group', 'know', 'castle_hill', 'comprise', 'nine', 'grant', 'prospecting_licence', 'project', 'area', 'cover', 'hectare', 'cazaly_registered', 'holder', 'castle_hill', 'project', 'tenement', 'cazalys', 'exploration', 'activity', 'report', 'period', 'direct_towards', 'acquisition', 'availa

In [23]:
# Sort documents by term frequency e.g. most frequency first
sorted_by_value = sorted(bow_corpus[0], key=lambda kv: kv[1], reverse=True)
print(sorted_by_value)

[(161, 27), (12, 17), (25, 13), (67, 13), (171, 10), (205, 10), (3, 8), (28, 7), (104, 7), (176, 7), (115, 6), (135, 6), (212, 6), (225, 6), (26, 5), (36, 5), (80, 5), (123, 5), (158, 5), (181, 5), (16, 4), (46, 4), (53, 4), (61, 4), (81, 4), (85, 4), (103, 4), (114, 4), (169, 4), (172, 4), (182, 4), (207, 4), (1, 3), (13, 3), (29, 3), (51, 3), (56, 3), (58, 3), (64, 3), (71, 3), (76, 3), (82, 3), (83, 3), (110, 3), (116, 3), (128, 3), (133, 3), (139, 3), (150, 3), (153, 3), (159, 3), (165, 3), (170, 3), (177, 3), (186, 3), (191, 3), (211, 3), (231, 3), (5, 2), (6, 2), (10, 2), (27, 2), (30, 2), (37, 2), (42, 2), (44, 2), (47, 2), (52, 2), (73, 2), (74, 2), (75, 2), (86, 2), (89, 2), (90, 2), (93, 2), (108, 2), (122, 2), (124, 2), (126, 2), (127, 2), (136, 2), (149, 2), (157, 2), (162, 2), (173, 2), (178, 2), (184, 2), (185, 2), (188, 2), (192, 2), (197, 2), (214, 2), (215, 2), (0, 1), (2, 1), (4, 1), (7, 1), (8, 1), (9, 1), (11, 1), (14, 1), (15, 1), (17, 1), (18, 1), (19, 1), (20, 1)

In [24]:
# View a human readable format of corpus (term-frequency)
[(id2word[id], freq) for id, freq in sorted_by_value] 

[('project', 27),
 ('area', 17),
 ('castle_hill', 13),
 ('exploration', 13),
 ('report', 10),
 ('tenement', 10),
 ('activity', 8),
 ('cazaly_resources', 7),
 ('kalgoorlie', 7),
 ('review', 7),
 ('limit', 6),
 ('nickel', 6),
 ('ultramafic', 6),
 ('within', 6),
 ('cazaly', 5),
 ('combine', 5),
 ('geology', 5),
 ('mafic', 5),
 ('potential', 5),
 ('september', 5),
 ('available', 4),
 ('coolgardie', 4),
 ('datum', 4),
 ('domain', 4),
 ('gold', 4),
 ('group', 4),
 ('july', 4),
 ('lie', 4),
 ('regional', 4),
 ('reporting', 4),
 ('sequence', 4),
 ('terrane', 4),
 ('acquisition', 3),
 ('assess', 3),
 ('cazalys', 3),
 ('dataset', 3),
 ('detail', 3),
 ('direct_towards', 3),
 ('eastern_goldfield', 3),
 ('fault', 3),
 ('follow', 3),
 ('grant', 3),
 ('greenstone', 3),
 ('kunanalle', 3),
 ('limited', 3),
 ('mineral', 3),
 ('nearby', 3),
 ('north', 3),
 ('part', 3),
 ('period', 3),
 ('previous', 3),
 ('rate', 3),
 ('relate', 3),
 ('rock', 3),
 ('sheet', 3),
 ('structural', 3),
 ('trend', 3),
 ('zone',

## Train LDA Model <a name="training"></a>

<img src="https://miro.medium.com/max/800/1*pZo_IcxW1GVuH2vQKdoIMQ.jpeg" alt="LDA example" style="width:600px;"/>

Here we train a LDA topic mode on our bag-of-words corpus. We initially set the `number of topics` we believe are in the corpus to 10. Only a single `pass` is performed through the corpus, but this could be increased to get better results. However, it is computationally expensive.

In [25]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=1,   # To get an improved LDA model, this can be increased
                                           alpha='auto',
                                           per_word_topics=True)

Here we show the topics elicited from the LDA model and the top keywords in the 10 topics. This shows the weighted importance of each word in each topic.

In [26]:
pprint(lda_model.print_topics())

[(0,
  '0.015*"area" + 0.014*"exploration" + 0.013*"tenement" + 0.012*"report" + '
  '0.011*"project" + 0.009*"sample" + 0.008*"australia" + 0.008*"western" + '
  '0.007*"within" + 0.007*"rock"'),
 (1,
  '0.017*"rock" + 0.015*"quartz" + 0.012*"metamorphic" + 0.012*"biotite" + '
  '0.011*"form" + 0.010*"small" + 0.010*"alteration" + 0.010*"grain" + '
  '0.009*"occur" + 0.008*"sulphide"'),
 (2,
  '0.033*"sbm" + 0.004*"sgw" + 0.004*"datum" + 0.004*"tem" + 0.003*"target" + '
  '0.003*"area" + 0.003*"report" + 0.003*"gem" + 0.003*"drill" + '
  '0.003*"client"'),
 (3,
  '0.015*"resource" + 0.011*"grade" + 0.010*"pit" + 0.009*"mining" + '
  '0.007*"report" + 0.007*"use" + 0.007*"deposit" + 0.006*"high" + '
  '0.006*"model" + 0.006*"datum"'),
 (4,
  '0.007*"carbine" + 0.007*"survey" + 0.006*"project" + 0.006*"aar" + '
  '0.006*"resource" + 0.005*"tenement" + 0.005*"sample" + 0.004*"drill" + '
  '0.004*"report" + 0.004*"exploration"'),
 (5,
  '0.027*"survey" + 0.016*"hole" + 0.013*"datum" + 0.0

In [27]:
doc_lda = lda_model[bow_corpus]

To evaluate the quality of our topic model, we can compute its `perplexity` and `coherence`.

In [28]:
# Compute Perplexity - lower is better.
print(f'Perplexity: {lda_model.log_perplexity(bow_corpus)}')

Perplexity: -8.502231255052806


In [29]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=bigram_corpus, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.48127630870109106


## Interactive LDA Visualisation <a name="visualisation"></a> 

#### Wait, what am I looking at again?

There are a lot of moving parts in the visualization. Here's a brief summary:

On the left, there is a plot of the "distance" between all of the topics (labeled as the Intertopic Distance Map)
The plot is rendered in two dimensions according a multidimensional scaling (MDS) algorithm. Topics that are generally similar should be appear close together on the plot, while dissimilar topics should appear far apart.

The relative size of a topic's circle in the plot corresponds to the relative frequency of the topic in the corpus.

An individual topic may be selected for closer scrutiny by clicking on its circle, or entering its number in the "selected topic" box in the upper-left.

On the right, there is a bar chart showing top terms.
When no topic is selected in the plot on the left, the bar chart shows the top most "salient" terms in the corpus. A term's saliency is a measure of both how frequent the term is in the corpus and how "distinctive" it is in distinguishing between different topics.

When a particular topic is selected, the bar chart changes to show the top most "relevant" terms for the selected topic. The relevance metric is controlled by the parameter λλ, which can be adjusted with a slider above the bar chart.

Setting the λλ parameter close to 1.0 (the default) will rank the terms solely according to their probability within the topic.

Setting λλ close to 0.0 will rank the terms solely according to their "distinctiveness" or "exclusivity" within the topic — i.e., terms that occur only in this topic, and do not occur in other topics.

Setting λλ to values between 0.0 and 1.0 will result in an intermediate ranking, weighting term probability and exclusivity accordingly.

Rolling the mouse over a term in the bar chart on the right will cause the topic circles to resize in the plot on the left, to show the strength of the relationship between the topics and the selected term.

A more detailed explanation of the pyLDAvis visualization can be found here. Unfortunately, though the data used by gensim and pyLDAvis are the same, they don't use the same ID numbers for topics. If you need to match up topics in gensim's LdaMulticore object and pyLDAvis' visualization, you have to dig through the terms manually.

#### Analyzing our LDA model
The interactive visualization pyLDAvis produces is helpful for both:

Better understanding and interpreting individual topics, and Better understanding the relationships between the topics.

For (1), you can manually select each topic to view its top most freqeuent and/or "relevant" terms, using different values of the λλ parameter. This can help when you're trying to assign a human interpretable name or "meaning" to each topic.

For (2), exploring the Intertopic Distance Plot can help you learn about how topics relate to each other, including potential higher-level structure between groups of topics.

#### Describing text with LDA
Beyond data exploration, one of the key uses for an LDA model is providing a compact, quantitative description of natural language text. Once an LDA model has been trained, it can be used to represent free text as a mixture of the topics the model learned from the original corpus. This mixture can be interpreted as a probability distribution across the topics, so the LDA representation of a paragraph of text might look like 50% Topic A, 20% Topic B, 20% Topic C, and 10% Topic D.

To use an LDA model to generate a vector representation of new text, you'll need to apply any text preprocessing steps you used on the model's training corpus to the new text, too. For our model, the preprocessing steps we used include:

Using spaCy to remove punctuation and lemmatize the text

- Applying our first-order phrase model to join word pairs
- Applying our second-order phrase model to join longer phrases
- Removing stopwords
- Creating a bag-of-words representation
- Once you've applied these preprocessing steps to the new text, it's ready to pass directly to the model to create an LDA representation. The lda_description(...) function will perform all these steps for us, including printing the resulting topical description of the input text.

##### Acknowledgement: https://www.kaggle.com/navinch/interesting-visualizations-lda-word2vec

### Visualize the topics

In [30]:
pyLDAvis.enable_notebook()

In [33]:
vis = pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, id2word)  # version 3.3+ use pyLDAvis.gensim_models.
vis

C:\Users\22917746\Miniconda3\envs\core-nlp-course\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.062982  0.035224       1        1  24.661038
7      0.104076 -0.003918       2        1  23.374521
3      0.022622  0.026841       3        1  15.260530
6      0.043616  0.016793       4        1  11.075219
5      0.053504  0.000142       5        1   9.616233
1      0.139441 -0.001239       6        1   8.419321
9      0.055857 -0.003247       7        1   4.707010
8     -0.218921  0.149857       8        1   2.680471
2     -0.155749 -0.130791       9        1   0.120455
4     -0.107429 -0.089664      10        1   0.085203, topic_info=          Term          Freq         Total Category  logprob  loglift
1083    survey   8930.000000   8930.000000  Default  30.0000  30.0000
12        area  16348.000000  16348.000000  Default  29.0000  29.0000
81        gold  14927.000000  14927.000000  Default  28.0000  28.0000
174   resource   7299.000000   7299.000000  Default  27.0000  27.0000
205   tenement  10044.000000  10044.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
222       west      2.831243   5161.833474  Topic10  -6.1685  -0.4404
910        pty      2.764270   4373.425620  Topic10  -6.1924  -0.2986
596      grade      2.772403   5432.172155  Topic10  -6.1895  -0.5125
135     nickel      2.695704   2849.834045  Topic10  -6.2175   0.1045
177       rock      2.741714  10334.571919  Topic10  -6.2006  -1.1668

[828 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
4051      1  0.616581            aar
4051      2  0.056053            aar
4051      3  0.056053            aar
4051      4  0.028026            aar
4051      5  0.028026            aar
...     ...       ...            ...
6496      3  0.028133  zuleika_shear
6496      5  0.056265  zuleika_shear
6496      6  0.028133  zuleika_shear
6496      7  0.028133  zuleika_shear
6496     10  0.028133  zuleika_shear

[3159 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 8, 4, 7, 6, 2, 10, 9, 3, 5])

## Additional Content: Tuning the number of topics (hyperparameter tuning) <a name="tuning"></a>

Here we explore how to find the best number of topics for our LDA model. This involves iteratively computing coherence values and finding an optimal topic distribution over a given number of topics.

⚠️ Computing coherence values can take a very long time to run.

Here we can visualise coherence scores as a function of the number of topics